Train Data-InputLayer-Hidden Layer - ReLU - Dropout - Output - SoftMax - Loss Function

1. Train Data
28*28사이즈 이미지를 784개의 1차원 벡터로 변환
\#flatten()

2. Hidden Layer
number of HL & nodes are critical hyperparameters for training.

3. ReLU
Kind of activation function.
If Input is larger than 0, the original value will be returned,
if not, it return 0.

모델에 비선형성을 부여해줌

4. Dropout
Function for decreasing the overfitting.
It removes several nodes of layer arbitrary following by given probability value.

5. Output Layer
Number of Layer should be 10, that is same with number of answers.

6.Softmax
It used when you make multi-class classification model.
it transfer the value of vector input between 0 to 1.

**??**
ReLU의 비선형성이 왜 필요하지?
그리고 DropOut과 같은 과적합 방지책이 또 있나? 왜 이런식으로 방지를 하지?




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np


train_dataset = datasets.MNIST(root='MNIST_data/',train = True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root = 'MNIST_data/', train = False,transform=transforms.ToTensor(), download=True)

print(len(train_dataset))



100%|██████████| 9912422/9912422 [00:00<00:00, 106252935.79it/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 29007589.52it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27135508.59it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14779308.59it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

60000


In [ ]:
#train용, validation용 분류
#validation : train된 데이터가 과적합 되었는지 확인하기 위함

train_dataset_size = int(len(train_dataset)*0.85)
validation_dataset_size = len(train_dataset)-train_dataset_size
train_dataset, validation_dataset = random_split(train_dataset,[train_dataset_size,validation_dataset_size])

print(len(train_dataset),len(validation_dataset))

51000 9000


In [ ]:
#한번에 트레이닝 시키는 묶음의 사이즈
BATCH_SIZE = 32

train_dataset_loader = DataLoader(dataset=train_dataset,batch_size = BATCH_SIZE,shuffle = True)
validation_dataset_loader = DataLoader(dataset = validation_dataset,batch_size = BATCH_SIZE,shuffle =True)
test_dataset_laoder = DataLoader(dataset = validation_dataset,batch_size = BATCH_SIZE, shuffle = True)


In [ ]:
class MyDeepLearningModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(784,256) #input : 28*28=784, output:256 why? 알수없음.
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.3)
    self.fc2 = nn.Linear(256,10)

  def forward(self, data):
    data = self.flatten(data)
    data = self.fc1(data)
    data = self.relu(data)
    data = self.dropout(data)
    logits = self.fc2(data)
    return logits

model = MyDeepLearningModel()

#uhm, 확률적인 데이터에 CEL쓰더라
loss_function = nn.CrossEntropyLoss() #model - loss function - optimizer관계가 어떻게 되는건지.. loss function을 모델이나 optimizer에 마운트 시킨적이 없는데 어떻게 loss function값을 기반으로 model parameter를 업데이트 한다는거지???????????

optimizer = torch.optim.SGD(model.parameters(),lr=1e-2)


In [ ]:
def model_train(dataloader,model,loss_function, optimizer):
  model.train()
  train_loss_sum = 0
  train_correct = 0
  train_total = 0

  total_train_batch = len(dataloader)

  for images, labels in dataloader:

    #-1 : 다른 차원으로 reshape 함을 뜻하는듯
    x_train = images.view(-1,28*28)
    y_train = labels #정답 숫자

    #output은 9개의 확률일텐데, 0~9숫자 자체인 y_train이랑 계산이 되나? CEL자체가 그런식으로 작동하는건가????????????????????????????????????????
    #nn.CrossEntropyLoss() 라는 손실함수 자체에 SoftMax알고리즘이 포함되어 있기때문에 가능..하다는건가
    #Softmax(소프트맥스)는 입력받은 값을 출력으로 0~1사이의 값으로 모두 정규화하며 출력 값들의 총합은 항상 1이 되는 특성을 가진 함수이다.
    #소프트맥스 결과값을 One hot encoder의 입력으로 연결하면 가장 큰 값만 True값, 나머지는 False값이 나오게 하여 이용 가능하다.
    output = model(x_train)
    loss = loss_function(output,y_train)

    #이거 왜하는걸까?
    #1배치 끝내고나서 그래디언트 초기화해주는것 같긴한데 일단 패스!
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss_sum += loss.item()

    train_total += y_train.size(0)
    #argmax(tensor,dimension), tensor.sum()==>tensor
    train_correct +=  ((torch.argmax(output,1)==y_train)).sum().item()

  train_avg_loss = train_loss_sum / total_train_batch
  train_avg_accuracy = 100*train_correct/train_total

  return (train_avg_loss, train_avg_accuracy)

def model_evaluate(dataloader, model, loss_function, optimizer):
  #모델을 평가모드로 전환 ==> Dropout, Batchnorm등의 기능을 비활성화, 메모리랑 상관X
  #훈련모드는 model.train()
  model.eval()

  with torch.no_grad(): #autogard engine비활성화, 메모리 leak 방지.
    val_loss_sum=0
    val_correct=0
    val_total =0

    total_val_batch = len(dataloader)

    for images, labels in dataloader :
      x_val = images.view(-1,28*28)
      y_val = labels

      output = model(x_val)
      loss = loss_function(output, y_val)

      val_loss_sum += loss.item()

      val_total += y_val.size(0)
      val_correct += (torch.argmax(output,1)==y_val).sum().item()

    val_avg_loss = val_loss_sum/total_val_batch
    val_avg_accuracy = 100*val_correct/val_total

  return (val_avg_loss,val_avg_accuracy)

def model_test(dataloader, model):
  model.eval()

  with torch.no_grad():
    test_loss_sum = 0
    test_correct = 0
    test_total = 0

    total_test_batch = len(dataloader)
    for images, labels in dataloader:

      x_test = images.view(-1,28*28)
      y_test = labels

      output = model(x_test)
      loss = loss_function(output, y_test)


      test_loss_sum += loss.item()

      test_total += y_test.size(0)
      test_correct += (torch.argmax(output,1)==y_test).sum().item()

    test_avg_loss = test_loss_sum / total_test_batch
    test_avg_accuracy = 100*test_correct / test_total

    print('accuracy', test_avg_accuracy)
    print('loss', test_avg_loss)

In [ ]:
from datetime import datetime

train_loss_list = []
train_accuracy_list = []

val_loss_list=[]
val_accuracy_list = []

start_time = datetime.now()

EPOCHS = 20

for epoch in range(EPOCHS):

  #model training
  train_avg_loss, train_avg_accuracy = model_train(train_dataset_loader,model,loss_function, optimizer)

  train_loss_list.append(train_avg_loss)
  train_accuracy_list.append(train_avg_accuracy)

  #model evaluation
  val_avg_loss, val_avg_accuracy = model_evaluate(validation_dataset_loader,model, loss_function,optimizer)

  val_loss_list.append(val_avg_loss)
  val_accuracy_list.append(val_avg_accuracy)

  print('epoch:', '%02d' % (epoch + 1),
          'train loss =', '{:.4f}'.format(train_avg_loss), 'train accuracy =', '{:.4f}'.format(train_avg_accuracy),
          'validation loss =', '{:.4f}'.format(val_avg_loss), 'validation accuracy =', '{:.4f}'.format(val_avg_accuracy))

end_time = datetime.now()

print('elapsed time => ', end_time-start_time)

epoch: 01 train loss = 0.0850 train accuracy = 97.5196 validation loss = 0.0910 validation accuracy = 97.3444
epoch: 02 train loss = 0.0839 train accuracy = 97.7000 validation loss = 0.0900 validation accuracy = 97.3556
epoch: 03 train loss = 0.0805 train accuracy = 97.7235 validation loss = 0.0895 validation accuracy = 97.3667
epoch: 04 train loss = 0.0805 train accuracy = 97.7118 validation loss = 0.0886 validation accuracy = 97.3667
epoch: 05 train loss = 0.0791 train accuracy = 97.7431 validation loss = 0.0885 validation accuracy = 97.4111
epoch: 06 train loss = 0.0774 train accuracy = 97.8020 validation loss = 0.0872 validation accuracy = 97.4667
epoch: 07 train loss = 0.0771 train accuracy = 97.7549 validation loss = 0.0866 validation accuracy = 97.4667
epoch: 08 train loss = 0.0739 train accuracy = 97.9196 validation loss = 0.0862 validation accuracy = 97.4222
epoch: 09 train loss = 0.0732 train accuracy = 97.8804 validation loss = 0.0850 validation accuracy = 97.5222
epoch: 10 